### Important:
The data used in this exercise has already been cleaned. However, I included this notebook with a basic data cleaning step, as the process may vary depending on the context. For example, if the token is a stablecoin, it might be necessary to remove very small or very large values. Zero-value transactions could be relevant for identifying network activity, or they might indicate failed transactions or contract issues. Repeated values in short time spans may suggest arbitrage or testing behaviour. Therefore, the cleaning strategy should be adapted to the analysis goal.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import pyarrow

input_csv = '../data/token_transfers_V3.0.0.csv'
output_csv = 'token_transfers_V3.0.0_cleaned.csv' # not used
output_parquet = 'token_transfers_V3.0.0_cleaned.parquet'

output_path = '../data/cleaned/'
os.makedirs(output_path, exist_ok=True)

df = pd.read_csv(input_csv)

df.info()
df.head()

In [ ]:
df_excluded = pd.DataFrame()

# standardise addresses in lower case
df['from_address'] = df['from_address'].str.lower()
df['to_address'] = df['to_address'].str.lower()
df['contract_address'] = df['contract_address'].str.lower()

# Simple validation - Ethereum
def is_valid_eth_address(addr):
    return isinstance(addr, str) and re.fullmatch(r"0x[a-f0-9]{40}", addr) is not None

# Rename timestamp
df.rename(columns={'time_stamp': 'timestamp'}, inplace=True)
df['timestamp'] = df['timestamp'].astype(int)

# Convert to date format
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date

# Remove lines with missing essential data
essential_cols = ['block_number', 'transaction_index', 'from_address', 'to_address', 'timestamp', 'contract_address', 'value']
missing_rows = df[df[essential_cols].isnull().any(axis=1)]
df_excluded = pd.concat([df_excluded, missing_rows])
df.drop(index=missing_rows.index, inplace=True)

# Remove negative values
negative_values = df[df['value'] < 0]
df_excluded = pd.concat([df_excluded, negative_values])
df.drop(index=negative_values.index, inplace=True)

# Remove invalid addresses
invalid_from = df[~df['from_address'].apply(is_valid_eth_address)]
invalid_to = df[~df['to_address'].apply(is_valid_eth_address)]
invalid_contract = df[~df['contract_address'].apply(is_valid_eth_address)]
invalid_rows = pd.concat([invalid_from, invalid_to, invalid_contract]).drop_duplicates()
df_excluded = pd.concat([df_excluded, invalid_rows])
df.drop(index=invalid_rows.index, inplace=True)

# final
n_total = len(df) + len(df_excluded)
print(f"Total original: {n_total}")
print(f"Valids: {len(df)}")
print(f"Excludeds: {len(df_excluded)}")
print(f"Tax of validation: {(len(df) / n_total) * 100:.2f}%")

In [ ]:
# Basic descriptive statistics
print("Descriptive:")
print(df['value'].describe())
print(f"\nPerccentiles: {df['value'].quantile([0.001, 0.01, 0.05, 0.95, 0.99, 0.999])}")

# Identify imposible values
test_values = df[df['value'] >= 1e15]
print(f"\nExtreme values (>= 1e15): {len(test_values)}")

# Analisys per contract
print("\nAnalisys per contract:")
for contract in df['contract_address'].value_counts().head(5).index:
    contract_data = df[df['contract_address'] == contract]
    print(f"{str(contract)[:10]}...: min={contract_data['value'].min():.4f}, max={contract_data['value'].max():.4f}")

# Remove outliers
q_high = df['value'].quantile(0.9999) # 99.99%
q_low = df['value'].quantile(0.0001) # 0.01%

outlier_high = df[df['value'] > q_high]
outlier_low = df[df['value'] < q_low]

print(f"\nHighest outliers (> {q_high:.2f}): {len(outlier_high)}")
print(f"Lowest outliers (< {q_low:.2f}): {len(outlier_low)}")

# Remove and update tracking
df_excluded = pd.concat([df_excluded, outlier_high, outlier_low])
df = df[(df['value'] >= q_low) & (df['value'] <= q_high)]



print(f"\nAfter outliers removal: {len(df)} valid transactions")

Direct links for mapping

USDT (Tether)
https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7

USDC (USD Coin)
https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48

DAI (Multi-Collateral DAI)
https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f

UST (TerraClassicUSD)
https://etherscan.io/token/0xa47c8bf37f92abed4a126bda807a7b7498661acd

PAX (Pax Dollar)
https://etherscan.io/token/0x8e870d67f660d95d5be530380d0ec0bd388289e1

WLUNA (Wrapped LUNA Classic)
https://etherscan.io/token/0xd2877702675e6ceb975b4a1dff9fb7baf4c91ea9

In [ ]:
# Map contract address to token names
contract_to_token = {
    '0xdac17f958d2ee523a2206206994597c13d831ec7': 'USDT',
    '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 'USDC',
    '0x6b175474e89094c44da98b954eedeac495271d0f': 'DAI',
    '0xa47c8bf37f92abed4a126bda807a7b7498661acd': 'UST',
    '0x8e870d67f660d95d5be530380d0ec0bd388289e1': 'PAX',
    '0xd2877702675e6ceb975b4a1dff9fb7baf4c91ea9': 'WLUNA'
}


df['stablecoin'] = df['contract_address'].map(contract_to_token)
df = df[~df['stablecoin'].isnull()]

print(f"Contract Addresses: {df['contract_address'].unique()}")
print(f"\nStablecoin: {df['stablecoin'].unique()}")


df.to_parquet(os.path.join(output_path, output_parquet), engine='pyarrow', index=False)

print()
df.info()
df.head()
